##### 它使用空白符号缩进和大量依赖外观的特色，特别适合用来表达或编辑数据结构、各种配置文件、倾印调试内容、文件大纲
我们用yaml做配置文件


In [3]:
import yaml #导入yaml包之后，就可以加载yaml文件并解析它
import os

In [4]:
#打开yaml文件，并获取yaml文件数据
file = open('./configs/hlcm.yaml', 'r',encoding="utf-8")
file_data =  file.read()


In [5]:
#查看file_data的类型
type(file_data)

str

In [6]:
#将字符串转换为字典或列表
data = yaml.safe_load(file_data)

##### 简单的一行，读取yaml文件，获取数据，并转换成字典

In [7]:
#简单的一行
data = yaml.safe_load(open('./configs/hlcm.yaml','r',encoding="utf-8"))
data

{'name': 'Household Location Choice Model',
 'model_type': 'segmented_discretechoice',
 'segmentation_col': 'income_quartile',
 'choosers_fit_filters': None,
 'choosers_predict_filters': None,
 'alts_fit_filters': ["general_type == 'Residential'"],
 'alts_predict_filters': ["general_type == 'Residential'"],
 'interaction_predict_filters': None,
 'choice_column': None,
 'sample_size': 100,
 'estimation_sample_size': 3000,
 'prediction_sample_size': None,
 'default_config': {'model_expression': 'np.log1p(residential_sales_price) + np.log1p(unit_sqft) + sum_residential_units + ave_unit_sqft + ave_lot_sqft + ave_income + hhsize + jobs + sfdu + renters + poor + population - 1'},
 'models': {0: {'fit_parameters': {'Coefficient': {'ave_income': -1.6387580142238778,
     'ave_lot_sqft': 0.058036758202680186,
     'ave_unit_sqft': -0.4108375616711735,
     'hhsize': -0.7344716467731971,
     'jobs': 0.0579594008099341,
     'np.log1p(residential_sales_price)': 0.985838370435962,
     'np.log1p(

In [8]:
#获取data里面的数据
data['alts_fit_filters']

["general_type == 'Residential'"]

In [9]:
data['default_config']  #字典

{'model_expression': 'np.log1p(residential_sales_price) + np.log1p(unit_sqft) + sum_residential_units + ave_unit_sqft + ave_lot_sqft + ave_income + hhsize + jobs + sfdu + renters + poor + population - 1'}

In [10]:
data['model_type']

'segmented_discretechoice'

In [11]:
from urbansim.models import RegressionModel, SegmentedRegressionModel, \
    MNLDiscreteChoiceModel, SegmentedMNLDiscreteChoiceModel, \
    GrowthRateTransition
def yaml_to_class(cfg):
    import yaml
    model_type = yaml.safe_load(open(cfg))["model_type"]
    return {
        "regression": RegressionModel,
        "segmented_regression": SegmentedRegressionModel,
        "discretechoice": MNLDiscreteChoiceModel,
        "segmented_discretechoice": SegmentedMNLDiscreteChoiceModel
    }[model_type]
yaml_to_class('./configs/hlcm.yaml')

urbansim.models.dcm.SegmentedMNLDiscreteChoiceModel

In [12]:
cfg = './configs/hlcm.yaml'

In [13]:
yaml_to_class(cfg).from_yaml(str_or_buffer = cfg)
#这个from_yaml 是urbansim.models.dcm.SegmentedMNLDiscreteChoiceModel的from_yaml()函数

##### from_yaml()从保存的yaml配置创建分段MNLCM 模型实例，参数是互斥的

In [14]:
cfg = './configs/hlcm.yaml'
yaml_to_class(cfg).from_yaml(str_or_buffer = cfg).columns_used()

['general_type',
 'residential_sales_price',
 'unit_sqft',
 'sum_residential_units',
 'ave_unit_sqft',
 'ave_lot_sqft',
 'ave_income',
 'hhsize',
 'jobs',
 'sfdu',
 'renters',
 'poor',
 'population',
 'income_quartile']

#### column_list()

In [15]:
import numpy as np
import pandas as pd
import toolz as tz
def column_list(tables, columns):
    """
    Take a list of tables and a list of column names and return the columns
    that are present in the tables.
    获取表列表和列名列表，并返回表中存在的列。
    Parameters
    ----------
    tables : sequence of _DataFrameWrapper or _TableFuncWrapper
        Could also be sequence of modified pandas.DataFrames, the important
        thing is that they have ``.name`` and ``.columns`` attributes.
    columns : sequence of str
        The column names of interest.
        感兴趣的列名
    Returns
    -------
    cols : list
        Lists of column names available in the tables.
        表中可用的列名
    """
    columns = set(columns)
    foundcols = tz.reduce(lambda x, y: x.union(y), (set(t.columns) for t in tables))
    return list(columns.intersection(foundcols))

df_households = pd.read_csv(r'D:\git_syy\pygeo-tutorial\佘越宇练习\区位选择模型\households_包含需要进行区位选择的家庭.csv')

cfg_columns =  ['general_type',
     'residential_sales_price',
     'unit_sqft',
     'sum_residential_units',
     'ave_unit_sqft',
     'ave_lot_sqft',
     'ave_income',
     'hhsize',
     'jobs',
     'sfdu',
     'renters',
     'poor',
     'population',
     'income_quartile']
print(df_households.columns)
column_list([df_households], cfg_columns) #处于df_households，又处于cfg中的

Index(['household_id', 'building_id', 'building_type_id', 'income', 'persons',
       'tenure', 'income_quartile', 'zone_id'],
      dtype='object')


['income_quartile']

In [16]:
tables = [df_households]
cfg_columns_used =  ['general_type',
     'residential_sales_price',
     'unit_sqft',
     'sum_residential_units',
     'ave_unit_sqft',
     'ave_lot_sqft',
     'ave_income',
     'hhsize',
     'jobs',
     'sfdu',
     'renters',
     'poor',
     'population',
     'income_quartile']
additional_columns = ["building_id"]
columns = column_list(tables, cfg_columns)+additional_columns
columns  #这个就是我们需要的列名（to_frame ）

['income_quartile', 'building_id']

In [17]:
import orca
tables
orca.add_table('households', df_households)

In [18]:
tables = [orca.get_table('households')]
tables  = [t for t in tables if t is not None]
len(tables)

1

In [19]:
tables[0].to_frame()  #这个就是 家庭表，households

,household_id,building_id,building_type_id,income,persons,tenure,income_quartile,zone_id
0,1,136325,1,27310,1,2,0,38.0
1,4,133982,2,25000,1,2,0,44.0
2,5,25633,1,3100,1,1,0,170.0
3,6,132596,2,7200,1,1,0,45.0
4,7,57108,1,27310,1,2,0,164.0
...,...,...,...,...,...,...,...,...
345583,2630612,74346,3,690,1,1,0,148.0
345584,2630613,78154,3,24650,1,1,0,141.0
345585,2630614,-1,3,58000,1,1,1,NaN
345586,2630615,137578,3,15400,1,1,0,22.0


In [20]:
columns = ['income_quartile', 'building_id']
df = tables[0].to_frame(columns)
df.to_csv('./chooser_df.csv')

In [21]:
df

,building_id,income_quartile
0,136325,0
1,133982,0
2,25633,0
3,132596,0
4,57108,0
...,...,...
345583,74346,0
345584,78154,0
345585,-1,1
345586,137578,0


#### 下面来找一下location_df 表
urbansim 制作的方式如下，一行一行看<br>
locations_df = to_frame([buildings, nodes], cfg,[supply_fname, vacant_fname])

In [22]:
buildings = pd.read_csv(r'D:\git_syy\pygeo-tutorial\佘越宇练习\区位选择模型\区位选择的基础建筑物表.csv',index_col=0)
nodes = pd.read_csv(r'D:\git_syy\pygeo-tutorial\佘越宇练习\区位选择模型\区位选择的基础表zones.csv',index_col=0)
households = pd.read_csv(r'D:\git_syy\pygeo-tutorial\佘越宇练习\区位选择模型\households_包含需要进行区位选择的家庭.csv',index_col=0)
choosers_df  = pd.read_csv('./chooser_df.csv',index_col=0)
print(nodes.columns)
nodes.columns=['gid', 'tract', 'area', 'acres', 'sum_residential_units',
       'sum_job_spaces', 'population', 'jobs', 'ave_lot_sqft', 'ave_income',
       'hhsize', 'ave_unit_sqft', 'sfdu', 'poor', 'renters', 'zone_id']
# print(nodes.columns)
print(nodes.columns)

Index(['gid', 'tract', 'area', 'acres', 'sum_residential_units',
       'sum_job_spaces', 'population', 'jobs', 'ave_lot_sqft', 'ave_income',
       'hhsize', 'ave_unit_sqft', 'sfdu', 'poor', 'renters', 'zone_id.1'],
      dtype='object')
Index(['gid', 'tract', 'area', 'acres', 'sum_residential_units',
       'sum_job_spaces', 'population', 'jobs', 'ave_lot_sqft', 'ave_income',
       'hhsize', 'ave_unit_sqft', 'sfdu', 'poor', 'renters', 'zone_id'],
      dtype='object')


In [23]:
orca.add_table('buildings',buildings)
orca.add_table('zones',nodes)
orca.add_table('households',households)
orca.add_table('choosers_df',choosers_df)  
# 既然打不败orca,那么就开始使用orca
# 打不败yaml，就用yaml，不要想着脱离yaml，和orca，除非后期有专业的计算机人才指导

In [24]:
import orca
# cfg = misc.config("hlcm.yaml")
choosers = orca.get_table('households')
nodes = orca.get_table('zones')
buildings = orca.get_table('buildings')
# choosers_df = orca.get_table('choosers_df')  

In [25]:
nodes.to_frame()

,gid,tract,area,acres,sum_residential_units,sum_job_spaces,population,jobs,ave_lot_sqft,ave_income,hhsize,ave_unit_sqft,sfdu,poor,renters,zone_id
zone_id,,,,,,,,,,,,,,,,
1442,525,126100.0,5272042.0,121.029,NaN,NaN,0.0,NaN,7.723413,11.060384,1.098612,7.103939,0.0,0.0,0.0,1442
1434,508,114100.0,7820341.5,179.530,NaN,NaN,0.0,NaN,7.723413,11.060384,1.098612,7.103939,0.0,0.0,0.0,1434
246,784,605000.0,4247867.5,97.518,NaN,NaN,0.0,NaN,7.723413,11.060384,1.098612,7.103939,0.0,0.0,0.0,246
1312,560,201800.0,759732500.0,17441.059,NaN,NaN,0.0,NaN,7.723413,11.060384,1.098612,7.103939,0.0,0.0,0.0,1312
705,1205,512507.0,27835386.0,639.013,NaN,NaN,0.0,NaN,7.723413,11.060384,1.098612,7.103939,0.0,0.0,0.0,705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,1446,NaN,23591500.0,541.586,NaN,NaN,0.0,NaN,7.723413,11.060384,1.098612,7.103939,0.0,0.0,0.0,769
874,1447,NaN,9820885.0,225.456,NaN,NaN,0.0,NaN,7.723413,11.060384,1.098612,7.103939,0.0,0.0,0.0,874
1146,1452,NaN,1179567.5,27.079,NaN,NaN,0.0,NaN,7.723413,11.060384,1.098612,7.103939,0.0,0.0,0.0,1146


In [26]:
[buildings, nodes]
nodes.columns

['gid',
 'tract',
 'area',
 'acres',
 'sum_residential_units',
 'sum_job_spaces',
 'population',
 'jobs',
 'ave_lot_sqft',
 'ave_income',
 'hhsize',
 'ave_unit_sqft',
 'sfdu',
 'poor',
 'renters',
 'zone_id']

##### cfg
cfg = misc.config('./yaml_syy_test.yaml')